In [ ]:
from itertools import product
from pathlib import Path
import numpy as np
from astropy.visualization import quantity_support
quantity_support()
import os

from spectrum_component_analyser.internals.spectrum import spectrum
from spectrum_component_analyser.internals.spectral_grid import spectral_grid
from spectrum_component_analyser.minimisation import calc_result, get_optimality, plot_nicely
from spectrum_component_analyser.internals.readers import read_HARPS_fits

external_spectrum_path = Path("../../assets/ADP.2016-02-04T01_02_52.843.fits")
script_dir = os.getcwd()  # usually the folder where notebook is running")
wavelength_grid_absolute_path = (script_dir / external_spectrum_path).resolve()

spectrum_to_decompose : spectrum = read_HARPS_fits(wavelength_grid_absolute_path, INTEGRATION_INDEX=0, verbose=False)
spectrum_to_decompose.plot()

mask = np.isfinite(spectrum_to_decompose.Fluxes)

spectrum_to_decompose = spectrum_to_decompose[mask]

print("reading in hdf5")
spectral_grid_relative_path = Path("../../assets/HARPS_convolved_spectral_grid.hdf5")
spectral_grid_absolute_path = (script_dir / spectral_grid_relative_path).resolve()
spec_grid : spectral_grid = spectral_grid.from_hdf5(absolute_path=spectral_grid_absolute_path)
lookup_table = spec_grid.to_lookup_table()

In [ ]:
from typing import Tuple
import astropy.units as u

all_parameters = list(product(spec_grid.T_effs, spec_grid.FeHs, spec_grid.Log_gs))

all_parameters = all_parameters[:50]

total_number_of_components = len(spec_grid.T_effs) * len(spec_grid.FeHs) * len(spec_grid.Log_gs)

A, result = calc_result(all_parameters, lookup_table, spec_grid=spec_grid, spectrum_to_decompose=spectrum_to_decompose, mask=mask, total_number_of_components=total_number_of_components)
print(f"residual MSE = {get_optimality(A, result, spectrum_to_decompose)}")

# from scipy.sparse.linalg import lsmr
# x = lsmr(A, spectrum_to_decompose.Fluxes.value)[0]
# x = np.clip(x, 0, 1)
# print(x)

In [ ]:
hash_map = plot_nicely(A, result, all_parameters, spec_grid, spectrum_to_decompose)